# TP02 — Forward Kinematics of Robot Manipulators

**GEII3 — Industrie 4.0: Robotique | Lab Session 2**

**Duration**: 2 sessions × 4 hours  
**Prerequisites**: Homogeneous transformations, rotation matrices, Python/NumPy basics

---

## Learning Objectives

By the end of this lab (8 hours total), you should be able to:

1. Apply **three different parameterization methods** for robot forward kinematics:
   - Standard Denavit-Hartenberg (DH)
   - Modified Denavit-Hartenberg (MDH / Craig's convention)
   - Elementary Transformations (Product of Exponentials - PoE)

2. Understand the **advantages and limitations** of each approach

3. Implement FK algorithms **from scratch** in Python

4. Validate implementations against **Robotics Toolbox** (Peter Corke)

5. Analyze **workspace, singularities**, and compare robot architectures

6. Gain practical experience with industrial robots: **2R planar, 3R anthropomorphic, SCARA, PUMA 560**

---

## Lab Organization

### **Session 1** (4 hours)
- **Part I**: Theory recap & helper functions (30 min)
- **Exercise 1**: 2R planar robot - all three methods (60 min)
- **Exercise 2**: 3-DOF anthropomorphic robot (90 min)
- **Exercise 3**: SCARA robot (60 min)

### **Session 2** (4 hours)
- **Exercise 4**: PUMA 560 (6-DOF industrial robot) (120 min)
- **Exercise 5**: Comparative analysis & methodology discussion (60 min)
- **Exercise 6**: Workspace analysis & singularities (60 min)

---

## Required Libraries

```python
numpy, matplotlib, roboticstoolbox-python, spatialmath-python, scipy
```

---

# Part I: Theoretical Background

## Three Approaches to Forward Kinematics

### 1. Standard Denavit-Hartenberg (DH) Convention

The **standard DH convention** (introduced in 1955) uses 4 parameters per link:

| Parameter | Description |
|-----------|-------------|
| $\theta_i$ | Joint angle: rotation about $z_{i-1}$ axis |
| $d_i$ | Link offset: translation along $z_{i-1}$ axis |
| $a_i$ | Link length: translation along $x_i$ axis |
| $\alpha_i$ | Link twist: rotation about $x_i$ axis |

**Transformation sequence**: $R_z(\theta_i) \cdot T_z(d_i) \cdot T_x(a_i) \cdot R_x(\alpha_i)$

**Transformation matrix**:
$$
{}^{i-1}T_i = \begin{bmatrix}
\cos\theta_i & -\sin\theta_i\cos\alpha_i & \sin\theta_i\sin\alpha_i & a_i\cos\theta_i \\
\sin\theta_i & \cos\theta_i\cos\alpha_i & -\cos\theta_i\sin\alpha_i & a_i\sin\theta_i \\
0 & \sin\alpha_i & \cos\alpha_i & d_i \\
0 & 0 & 0 & 1
\end{bmatrix}
$$

**Advantages**: Widely used, compact representation  
**Disadvantages**: Frame assignment can be tricky; issues with parallel axes

---

### 2. Modified Denavit-Hartenberg (MDH / Craig's Convention)

Same 4 parameters but **different frame attachment** (frames attached to joints, not links):

**Transformation sequence**: $R_x(\alpha_{i-1}) \cdot T_x(a_{i-1}) \cdot R_z(\theta_i) \cdot T_z(d_i)$

**Transformation matrix**:
$$
{}^{i-1}T_i = \begin{bmatrix}
\cos\theta_i & -\sin\theta_i & 0 & a_{i-1} \\
\sin\theta_i\cos\alpha_{i-1} & \cos\theta_i\cos\alpha_{i-1} & -\sin\alpha_{i-1} & -d_i\sin\alpha_{i-1} \\
\sin\theta_i\sin\alpha_{i-1} & \cos\theta_i\sin\alpha_{i-1} & \cos\alpha_{i-1} & d_i\cos\alpha_{i-1} \\
0 & 0 & 0 & 1
\end{bmatrix}
$$

**Advantages**: Frame $i$ attached to joint $i$ (more intuitive); used by RTB  
**Disadvantages**: Less common in textbooks; different from standard DH

---

### 3. Product of Exponentials (PoE) / Elementary Transformations

Based on **screw theory** and Lie algebra. Each joint contributes a **twist**:

$$
T(q) = e^{[\mathcal{S}_1]q_1} e^{[\mathcal{S}_2]q_2} \cdots e^{[\mathcal{S}_n]q_n} M
$$

Where:
- $\mathcal{S}_i = (\omega_i, v_i)$ is the **screw axis** for joint $i$
  - $\omega_i \in \mathbb{R}^3$: rotation axis (unit vector)
  - $v_i \in \mathbb{R}^3$: linear velocity (for revolute: $v = \omega \times q$, where $q$ is a point on axis)
- $M \in SE(3)$: **home configuration** (all joint variables = 0)
- $e^{[\mathcal{S}]q}$: matrix exponential

**Advantages**: 
- Geometrically intuitive (just specify axis and home position)
- No singularities in parameterization
- Easy to modify robot structure
- Modern approach (used in research, MoveIt!, modern textbooks)

**Disadvantages**: 
- Requires matrix exponential computation
- Less common in industrial practice

---

## Setup and Installation

In [ ]:
# Uncomment to install (takes ~2 minutes):
# !pip install -q roboticstoolbox-python spatialmath-python matplotlib scipy

import numpy as np
import matplotlib.pyplot as plt
from math import cos, sin, pi, atan2, sqrt
from scipy.linalg import expm

# Import Robotics Toolbox
try:
    import roboticstoolbox as rtb
    from spatialmath import SE3
    RTB_AVAILABLE = True
    print("✓ Robotics Toolbox loaded successfully")
    print(f"  RTB version: {rtb.__version__}")
except ImportError:
    RTB_AVAILABLE = False
    print("⚠ Robotics Toolbox not available. Run installation cell above.")

np.set_printoptions(precision=4, suppress=True)
print(f"✓ NumPy version: {np.__version__}")
print(f"✓ Python libraries ready!\n")

## Helper Functions

Implement the basic functions for all three conventions.

In [ ]:
# ============================================================================
# STANDARD DH CONVENTION
# ============================================================================

def dh_standard(theta, d, a, alpha):
    """
    Standard DH transformation matrix.
    Order: Rz(theta) * Tz(d) * Tx(a) * Rx(alpha)
    
    Parameters:
    - theta: rotation about z_{i-1}
    - d: translation along z_{i-1}
    - a: translation along x_i
    - alpha: rotation about x_i
    
    Returns: 4x4 homogeneous transformation matrix
    """
    ct, st = cos(theta), sin(theta)
    ca, sa = cos(alpha), sin(alpha)
    
    T = np.array([
        [ct,           -st*ca,           st*sa,           a*ct],
        [st,            ct*ca,          -ct*sa,           a*st],
        [0,                sa,              ca,              d],
        [0,                 0,               0,              1]
    ])
    return T


# ============================================================================
# MODIFIED DH CONVENTION (Craig's convention)
# ============================================================================

def dh_modified(theta, d, a, alpha):
    """
    Modified DH transformation matrix (Craig's convention).
    Order: Rx(alpha_{i-1}) * Tx(a_{i-1}) * Rz(theta_i) * Tz(d_i)
    
    Parameters:
    - theta: rotation about z_i
    - d: translation along z_i
    - a: translation along x_{i-1}
    - alpha: rotation about x_{i-1}
    
    Returns: 4x4 homogeneous transformation matrix
    """
    ct, st = cos(theta), sin(theta)
    ca, sa = cos(alpha), sin(alpha)
    
    T = np.array([
        [ct,           -st,              0,               a],
        [st*ca,         ct*ca,          -sa,           -d*sa],
        [st*sa,         ct*sa,           ca,            d*ca],
        [0,                 0,            0,               1]
    ])
    return T


# ============================================================================
# ELEMENTARY TRANSFORMATIONS (Product of Exponentials)
# ============================================================================

def skew_symmetric(v):
    """Convert 3D vector to skew-symmetric matrix [v]."""
    return np.array([
        [0,    -v[2],  v[1]],
        [v[2],     0, -v[0]],
        [-v[1], v[0],     0]
    ])


def twist_to_matrix(omega, v):
    """
    Convert twist (omega, v) to 4x4 matrix form [S].
    
    [S] = [  [omega]   v ]
          [    0      0 ]
    """
    S = np.zeros((4, 4))
    S[:3, :3] = skew_symmetric(omega)
    S[:3, 3] = v
    return S


def screw_to_transform(screw, q):
    """
    Compute matrix exponential e^{[S]q}.
    
    Parameters:
    - screw: (omega, v) tuple representing screw axis
      omega: 3D rotation axis (unit vector for revolute)
      v: linear part (v = omega x r for revolute, where r is point on axis)
    - q: joint variable (angle in rad for revolute, distance for prismatic)
    
    Returns: 4x4 transformation matrix
    """
    omega, v = screw
    S_mat = twist_to_matrix(omega, v)
    return expm(S_mat * q)


# ============================================================================
# GENERIC FORWARD KINEMATICS
# ============================================================================

def fk_dh(dh_params, q, convention='standard'):
    """
    Compute forward kinematics using DH parameters.
    
    Parameters:
    - dh_params: list of tuples (theta_offset, d_offset, a, alpha, joint_type)
                 joint_type: 'r' (revolute) or 'p' (prismatic)
    - q: array of joint values [q1, q2, ..., qn]
    - convention: 'standard' or 'modified'
    
    Returns: 
    - T_0n: end-effector transformation (4x4 matrix)
    - T_list: list of intermediate transformations [T_0, T_01, T_02, ...]
    """
    dh_func = dh_standard if convention == 'standard' else dh_modified
    
    T = np.eye(4)
    T_list = [T.copy()]
    
    for i, (theta_off, d_off, a, alpha, jtype) in enumerate(dh_params):
        if jtype == 'r':  # revolute
            theta = theta_off + q[i]
            d = d_off
        else:  # prismatic
            theta = theta_off
            d = d_off + q[i]
        
        T_i = dh_func(theta, d, a, alpha)
        T = T @ T_i
        T_list.append(T.copy())
    
    return T, T_list


def fk_poe(screws, q, M):
    """
    Compute forward kinematics using Product of Exponentials.
    
    Parameters:
    - screws: list of (omega, v) tuples for each joint
    - q: array of joint values
    - M: home configuration (4x4 matrix, q = [0, 0, ...])
    
    Returns: T(q) = e^{[S1]q1} * e^{[S2]q2} * ... * e^{[Sn]qn} * M
    """
    T = np.eye(4)
    for screw, qi in zip(screws, q):
        T = T @ screw_to_transform(screw, qi)
    return T @ M


# ============================================================================
# VISUALIZATION
# ============================================================================

def plot_robot_2d(T_list, ax=None, title="Robot Configuration", draw_frames=False):
    """
    Visualize robot in 2D (XY projection).
    
    Parameters:
    - T_list: list of transformation matrices from base to each joint
    - ax: matplotlib axis (creates new if None)
    - title: plot title
    - draw_frames: whether to draw coordinate frames at each joint
    """
    if ax is None:
        fig, ax = plt.subplots(figsize=(8, 8))
    
    # Extract positions
    x = [T[0, 3] for T in T_list]
    y = [T[1, 3] for T in T_list]
    
    # Draw links
    ax.plot(x, y, 'o-', linewidth=2, markersize=8, label='Robot', color='blue')
    ax.plot(x[0], y[0], 'gs', markersize=12, label='Base')
    ax.plot(x[-1], y[-1], 'r*', markersize=15, label='End-effector')
    
    # Draw coordinate frames if requested
    if draw_frames:
        scale = 0.08
        for i, T in enumerate(T_list):
            # X-axis (red)
            ax.arrow(T[0,3], T[1,3], scale*T[0,0], scale*T[1,0], 
                     head_width=0.03, head_length=0.02, fc='red', ec='red', alpha=0.6)
            # Y-axis (green)
            ax.arrow(T[0,3], T[1,3], scale*T[0,1], scale*T[1,1], 
                     head_width=0.03, head_length=0.02, fc='green', ec='green', alpha=0.6)
    else:
        # Just end-effector frame
        T_end = T_list[-1]
        scale = 0.12
        ax.arrow(T_end[0,3], T_end[1,3], scale*T_end[0,0], scale*T_end[1,0], 
                 head_width=0.04, head_length=0.03, fc='red', ec='red', alpha=0.7)
        ax.arrow(T_end[0,3], T_end[1,3], scale*T_end[0,1], scale*T_end[1,1], 
                 head_width=0.04, head_length=0.03, fc='green', ec='green', alpha=0.7)
    
    ax.grid(True, alpha=0.3)
    ax.axis('equal')
    ax.set_xlabel('X (m)', fontsize=11)
    ax.set_ylabel('Y (m)', fontsize=11)
    ax.set_title(title, fontsize=12, fontweight='bold')
    ax.legend(loc='best')
    
    return ax


def compare_with_rtb(T_manual, T_rtb, tol=1e-4, verbose=True):
    """
    Compare manually computed matrix with RTB.
    """
    if isinstance(T_rtb, SE3):
        T_rtb = T_rtb.A
    
    diff = np.abs(T_manual - T_rtb)
    max_diff = np.max(diff)
    
    if verbose:
        print("\n" + "="*50)
        print("   Comparison: Manual Implementation vs RTB")
        print("="*50)
        print(f"Maximum difference: {max_diff:.2e}")
    
    if max_diff < tol:
        if verbose:
            print("✓ Results match! Your implementation is correct.")
        return True
    else:
        if verbose:
            print("✗ Significant differences detected")
            print("\nManual Matrix:")
            print(T_manual)
            print("\nRTB Matrix:")
            print(T_rtb)
            print("\nDifference:")
            print(diff)
        return False


def extract_position(T):
    """Extract position vector from transformation matrix."""
    return T[:3, 3]


def extract_rotation(T):
    """Extract rotation matrix from transformation matrix."""
    return T[:3, :3]


def rotation_to_euler_zyx(R):
    """Convert rotation matrix to ZYX Euler angles (in radians)."""
    if abs(R[2,0]) != 1:
        theta_y = -np.arcsin(R[2,0])
        theta_z = np.arctan2(R[1,0]/np.cos(theta_y), R[0,0]/np.cos(theta_y))
        theta_x = np.arctan2(R[2,1]/np.cos(theta_y), R[2,2]/np.cos(theta_y))
    else:
        theta_z = 0
        if R[2,0] == -1:
            theta_y = np.pi/2
            theta_x = theta_z + np.arctan2(R[0,1], R[0,2])
        else:
            theta_y = -np.pi/2
            theta_x = -theta_z + np.arctan2(-R[0,1], -R[0,2])
    return np.array([theta_z, theta_y, theta_x])

print("✓ Helper functions defined successfully\n")

---

# Exercise 1: 2R Planar Robot (All Three Methods)

**Estimated time**: 60 minutes

## Problem Description

Consider a simple **2-DOF planar robot** (2R) with:
- Two revolute joints rotating about vertical (Z) axes
- Link lengths: $L_1 = 0.4$ m, $L_2 = 0.3$ m
- All motion in the XY plane

## Task 1.1: Standard DH Table (15 min)

**Instructions**:
1. Draw the robot schematically with coordinate frames following **standard DH convention**
2. Complete the DH parameter table
3. Write the transformation matrices $T_0^1$ and $T_1^2$
4. Derive $T_0^2$ symbolically

### Your derivation:

**Sketch** (insert image or describe frame placement):

_[Draw or describe here]_

**DH Table** (Standard):

| Joint i | $\theta_i$ | $d_i$ | $a_i$ | $\alpha_i$ |
|---------|-----------|-------|-------|------------|
| 1       | ?         | ?     | ?     | ?          |
| 2       | ?         | ?     | ?     | ?          |

**Matrix expressions**:

$T_0^1 = ?$

$T_1^2 = ?$

$T_0^2 = T_0^1 \cdot T_1^2 = ?$

---

## Task 1.2: Implementation with Standard DH (10 min)

In [ ]:
# Robot parameters
L1 = 0.4  # m
L2 = 0.3  # m

# DH parameters for 2R robot (Standard convention)
# Format: (theta_offset, d_offset, a, alpha, joint_type)
# TODO: Complete according to your table
dh_2r_standard = [
    (0,  0,  L1,  0,  'r'),  # Joint 1 - VERIFY THIS
    (0,  0,  L2,  0,  'r'),  # Joint 2 - VERIFY THIS
]

# Test configuration: q1 = 30°, q2 = 45°
q_2r = np.array([np.deg2rad(30), np.deg2rad(45)])

# Compute FK with standard DH
T_2r_std, T_list_std = fk_dh(dh_2r_standard, q_2r, convention='standard')

print("\n" + "="*60)
print("  2R ROBOT - STANDARD DH")
print("="*60)
print(f"Configuration: q = [{np.rad2deg(q_2r[0]):.1f}°, {np.rad2deg(q_2r[1]):.1f}°]")
print(f"\nEnd-effector position:")
print(f"  x = {T_2r_std[0,3]:.4f} m")
print(f"  y = {T_2r_std[1,3]:.4f} m")
print(f"  z = {T_2r_std[2,3]:.4f} m")
print(f"\nFull transformation matrix T_0^2:")
print(T_2r_std)

# Visualize
fig, ax = plt.subplots(figsize=(8, 8))
plot_robot_2d(T_list_std, ax, 
              title=f"2R Robot (Standard DH)\nq = {np.rad2deg(q_2r)}°",
              draw_frames=True)
plt.show()

## Task 1.3: Modified DH Table and Implementation (15 min)

**Instructions**:
1. Establish coordinate frames following **modified DH convention** (frames attached to joints)
2. Complete the MDH parameter table
3. Implement and verify

### Your derivation:

**Modified DH Table**:

| Joint i | $\theta_i$ | $d_i$ | $a_{i-1}$ | $\alpha_{i-1}$ |
|---------|-----------|-------|-----------|----------------|
| 1       | ?         | ?     | ?         | ?              |
| 2       | ?         | ?     | ?         | ?              |

**Note**: Pay attention to the indexing! In MDH, $a_{i-1}$ and $\alpha_{i-1}$ refer to the **previous** link.

---

In [ ]:
# DH parameters for 2R robot (Modified convention)
# Format: (theta_offset, d_offset, a, alpha, joint_type)
# TODO: Complete according to your MDH table
dh_2r_modified = [
    (0,  0,  0,   0,  'r'),  # Joint 1 - a_{0}, alpha_{0}
    (0,  0,  L1,  0,  'r'),  # Joint 2 - a_{1}, alpha_{1}
]

# Compute FK with modified DH
T_2r_mdh, T_list_mdh = fk_dh(dh_2r_modified, q_2r, convention='modified')

print("\n" + "="*60)
print("  2R ROBOT - MODIFIED DH")
print("="*60)
print(f"Configuration: q = [{np.rad2deg(q_2r[0]):.1f}°, {np.rad2deg(q_2r[1]):.1f}°]")
print(f"\nEnd-effector position:")
print(f"  x = {T_2r_mdh[0,3]:.4f} m")
print(f"  y = {T_2r_mdh[1,3]:.4f} m")
print(f"  z = {T_2r_mdh[2,3]:.4f} m")
print(f"\nFull transformation matrix T_0^2:")
print(T_2r_mdh)

# Compare with standard DH
print("\n--- Comparison: Standard vs Modified DH ---")
diff = np.max(np.abs(T_2r_std - T_2r_mdh))
print(f"Maximum difference: {diff:.2e}")
if diff < 1e-10:
    print("✓ Both methods give the same result!")
else:
    print("✗ Results differ - check your parameters")

## Task 1.4: Product of Exponentials (20 min)

**Instructions**:
1. Define the **home configuration** $M$ (when $q_1 = q_2 = 0$)
2. Identify the **screw axes** for each joint:
   - Joint 1: rotation axis $\omega_1$, point on axis $r_1$, compute $v_1 = \omega_1 \times r_1$
   - Joint 2: rotation axis $\omega_2$, point on axis $r_2$, compute $v_2 = \omega_2 \times r_2$
3. Implement using PoE formula

### Your derivation:

**Home configuration** $M$ (when all joints = 0):

$M = ?$ (hint: for 2R planar, end-effector is at $(L_1 + L_2, 0, 0)$ with identity rotation)

**Screw axes**:

Joint 1:
- Rotation axis: $\omega_1 = ?$ (hint: both joints rotate about Z)
- Point on axis: $r_1 = ?$ (hint: joint 1 is at origin)
- Linear part: $v_1 = \omega_1 \times r_1 = ?$

Joint 2:
- Rotation axis: $\omega_2 = ?$
- Point on axis: $r_2 = ?$ (hint: joint 2 is at $(L_1, 0, 0)$ in home config)
- Linear part: $v_2 = \omega_2 \times r_2 = ?$

---

In [ ]:
# Define home configuration M (q = [0, 0])
# TODO: Complete this
M_2r = np.array([
    [1, 0, 0, L1+L2],  # End-effector at (L1+L2, 0, 0)
    [0, 1, 0, 0],
    [0, 0, 1, 0],
    [0, 0, 0, 1]
])

# Define screw axes
# TODO: Complete these based on your derivation
omega1 = np.array([0, 0, 1])  # Rotation about Z
r1 = np.array([0, 0, 0])       # Joint 1 at origin
v1 = np.cross(omega1, r1)      # v = omega x r

omega2 = np.array([0, 0, 1])   # Rotation about Z  
r2 = np.array([L1, 0, 0])      # Joint 2 at (L1, 0, 0) in home config
v2 = np.cross(omega2, r2)      # v = omega x r

screws_2r = [
    (omega1, v1),
    (omega2, v2)
]

print("\nScrew axes:")
print(f"S1 = (omega: {omega1}, v: {v1})")
print(f"S2 = (omega: {omega2}, v: {v2})")

# Compute FK using PoE
T_2r_poe = fk_poe(screws_2r, q_2r, M_2r)

print("\n" + "="*60)
print("  2R ROBOT - PRODUCT OF EXPONENTIALS")
print("="*60)
print(f"Configuration: q = [{np.rad2deg(q_2r[0]):.1f}°, {np.rad2deg(q_2r[1]):.1f}°]")
print(f"\nEnd-effector position:")
print(f"  x = {T_2r_poe[0,3]:.4f} m")
print(f"  y = {T_2r_poe[1,3]:.4f} m")
print(f"  z = {T_2r_poe[2,3]:.4f} m")
print(f"\nFull transformation matrix:")
print(T_2r_poe)

# Compare all three methods
print("\n" + "="*60)
print("  COMPARISON: All Three Methods")
print("="*60)
diff_std_poe = np.max(np.abs(T_2r_std - T_2r_poe))
diff_mdh_poe = np.max(np.abs(T_2r_mdh - T_2r_poe))
print(f"Standard DH vs PoE: max diff = {diff_std_poe:.2e}")
print(f"Modified DH vs PoE: max diff = {diff_mdh_poe:.2e}")

if diff_std_poe < 1e-10 and diff_mdh_poe < 1e-10:
    print("\n✓ Excellent! All three methods give identical results.")
else:
    print("\n✗ Discrepancies detected - review your derivations.")

## Task 1.5: Validation with RTB (10 min)

In [ ]:
if RTB_AVAILABLE:
    from roboticstoolbox import DHRobot, RevoluteDH
    
    # Create 2R robot using RTB (uses Modified DH by default)
    robot_2r = DHRobot([
        RevoluteDH(a=0,  d=0, alpha=0),
        RevoluteDH(a=L1, d=0, alpha=0)
    ], name="2R Planar")
    
    print(robot_2r)
    
    # Compute FK with RTB
    T_rtb = robot_2r.fkine(q_2r)
    
    print("\n" + "="*60)
    print("  RTB Forward Kinematics")
    print("="*60)
    print(T_rtb)
    
    # Compare with our implementations
    print("\n--- Validation ---")
    compare_with_rtb(T_2r_std, T_rtb, verbose=True)
    compare_with_rtb(T_2r_poe, T_rtb, verbose=True)
    
    # Visualize with RTB (optional)
    # robot_2r.plot(q_2r, backend='pyplot', block=False)
    
else:
    print("RTB not available - skipping validation")

## Task 1.6: Analysis Questions (10 min)

Answer the following questions based on your experience with this exercise:

### Q1: Which method was easiest to apply for this simple 2R robot? Why?

_Your answer:_


### Q2: What are the main differences between Standard and Modified DH in terms of frame placement?

_Your answer:_


### Q3: For the PoE method, what is the physical meaning of the vector $v_i$?

_Your answer:_


### Q4: If we add a third revolute joint at the end-effector, which method would require the least modification? Why?

_Your answer:_


---